In [25]:
# Imports
from tensorflow.keras.layers import TextVectorization
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from sklearn.model_selection import RepeatedKFold, cross_val_score
from tensorflow.keras.models import * 
from tensorflow.keras.layers import * 
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold

In [26]:
# Import data
data = pd.read_csv("../../data/mturk_experiment_2.csv",encoding='unicode_escape')
labels = data["Formality"]
samples = data["Sentence"]


train_samples, test_samples, train_labels,test_labels = train_test_split(samples, labels, test_size=0.2)

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [27]:
vectoriser = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectoriser.adapt(text_ds)

voc = vectoriser.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

glove_path = "glove.6B.300d.txt"
embeddings_index = {}
with open(glove_path,encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs,"f",sep= " ")
        embeddings_index[word] = coefs

num_tokens = len(voc) + 2
embedding_dim = 300
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens,embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

        
embedding_layer = Embedding(num_tokens,embedding_dim,embeddings_initializer=keras.initializers.Constant(embedding_matrix),trainable=False)

In [28]:
def buildmodel():

    int_sequences_input = keras.Input(shape=(None,), dtype="int64")
    embedded_sequences = embedding_layer(int_sequences_input)
    x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
    x = layers.MaxPooling1D(1)(x)
    x = layers.Conv1D(128, 1, activation="relu")(x)
    x = layers.MaxPooling1D(1)(x)
    x = layers.Conv1D(128, 1, activation="relu")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    preds = layers.Dense(units=1)(x)
    model = keras.Model(int_sequences_input, preds)
    model.compile(optimizer=Adam(learning_rate=0.001),loss='mean_squared_error',metrics=[tf.keras.metrics.MeanSquaredError(),tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])     
    return model

In [29]:
x_train = vectoriser(np.array([[s] for s in train_samples])).numpy()
x_val = vectoriser(np.array([[s] for s in test_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(test_labels)

In [39]:
model = buildmodel()
model.fit(x=x_train,y=y_train,batch_size=1,epochs=15)

Epoch 1/15
5625/5625 [==============================] - 23s 4ms/step - loss: 1.3028 - mean_squared_error: 1.3028 - mean_absolute_error: 0.9021 - mean_absolute_percentage_error: 23.9512
Epoch 2/15
5625/5625 [==============================] - 19s 3ms/step - loss: 0.9888 - mean_squared_error: 0.9888 - mean_absolute_error: 0.7955 - mean_absolute_percentage_error: 21.3779
Epoch 3/15
5625/5625 [==============================] - 20s 4ms/step - loss: 0.8670 - mean_squared_error: 0.8670 - mean_absolute_error: 0.7449 - mean_absolute_percentage_error: 20.2457
Epoch 4/15
5625/5625 [==============================] - 19s 3ms/step - loss: 0.7610 - mean_squared_error: 0.7610 - mean_absolute_error: 0.7011 - mean_absolute_percentage_error: 19.1100
Epoch 5/15
5625/5625 [==============================] - 20s 3ms/step - loss: 0.6804 - mean_squared_error: 0.6804 - mean_absolute_error: 0.6636 - mean_absolute_percentage_error: 18.1109
Epoch 6/15
5625/5625 [==============================] - 19s 3ms/step - loss

In [40]:
scores = model.evaluate(x=x_val,y=y_val)
scores[1:]

44/44 [==============================] - 0s 7ms/step - loss: 0.6757 - mean_squared_error: 0.6757 - mean_absolute_error: 0.6653 - mean_absolute_percentage_error: 18.4747


[0.6756930351257324, 0.6652719974517822, 18.47465705871582]

In [32]:
'''
estimator= KerasRegressor(build_fn=buildmodel, epochs=20, batch_size=32, verbose=1)
cv = KFold(n_splits=10, random_state=1, shuffle=True)
kfold= RepeatedKFold(n_splits=5, n_repeats=100)
results= cross_val_score(estimator, x_train, y_train, cv=cv, n_jobs=-1)  # 2 cpus
results.mean()  # Mean MSE
'''

'\nestimator= KerasRegressor(build_fn=buildmodel, epochs=20, batch_size=32, verbose=1)\ncv = KFold(n_splits=10, random_state=1, shuffle=True)\nkfold= RepeatedKFold(n_splits=5, n_repeats=100)\nresults= cross_val_score(estimator, x_train, y_train, cv=cv, n_jobs=-1)  # 2 cpus\nresults.mean()  # Mean MSE\n'